# Recurrent Neural Network 

A neural network usually takes an independent variable X (or a set of independent variables ) and a dependent variable y then it learns the mapping between X and y (we call this Training), Once training is done , we give a new independent variable to predict the dependent variable.


Recurrent Neural Networks (RNN) are very effective for Natural Language Processing and other sequence tasks because they have "memory". They can read inputs $x^{\langle t \rangle}$ (such as words) one at a time, and remember some information/context through the hidden layer activations that get passed from one time-step to the next. This allows a uni-directional RNN to take information from the past to process later inputs. A bidirection RNN can take context from both the past and the future. 

**Notation**:
- Superscript $[l]$ denotes an object associated with the $l^{th}$ layer. 
    - Example: $a^{[4]}$ is the $4^{th}$ layer activation. $W^{[5]}$ and $b^{[5]}$ are the $5^{th}$ layer parameters.

- Superscript $(i)$ denotes an object associated with the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example input.

- Superscript $\langle t \rangle$ denotes an object at the $t^{th}$ time-step. 
    - Example: $x^{\langle t \rangle}$ is the input x at the $t^{th}$ time-step. $x^{(i)\langle t \rangle}$ is the input at the $t^{th}$ timestep of example $i$.
    
- Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $a^{[l]}_i$ denotes the $i^{th}$ entry of the activations in layer $l$.



## 1 - Forward propagation for the basic Recurrent Neural Network

Later this week, you will generate music using an RNN. The basic RNN that you will implement has the structure below. In this example, $T_x = T_y$. 

<img src="images/RNN.png" style="width:500;height:300px;">
<caption><center> **Figure 1**: Basic RNN model </center></caption>

Here's how you can implement an RNN: 

**Steps**:
1. Implement the calculations needed for one time-step of the RNN.
2. Implement a loop over $T_x$ time-steps in order to process all the inputs, one at a time. 
$T_x$ is just a length of your sequence

Let's go!

## 1.1 - RNN cell

A Recurrent neural network can be seen as the repetition of a single cell. You are first going to implement the computations for a single time-step. The following figure describes the operations for a single time-step of an RNN cell. 

<img src="images/rnn_step_forward.png" style="width:700px;height:300px;">
<caption><center> **Figure 2**: Basic RNN cell. Takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $y^{\langle t \rangle}$ </center></caption>

1. Compute the hidden state with tanh activation: $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
2. Using your new hidden state $a^{\langle t \rangle}$, compute the prediction $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. We provided you a function: `softmax`.
3. Store $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)$ in cache
4. Return $a^{\langle t \rangle}$ , $y^{\langle t \rangle}$ and cache



# Data preparing

In [1]:
import numpy as np
import pandas as pd

In [2]:
# lets we have four sentences 
s1 = 'mango is yellow color'
s2 = 'banana is pink color'
s3 = 'hair has black color'

# Now what is and how the matrices shapes define
# create vocab
l = [*s1.split(),*s2.split(),*s3.split()]
l.append('<end>')
vocab = sorted(set(l))
one_hot_vector_vocab = np.array(pd.get_dummies(vocab))
print(pd.get_dummies(vocab))

   <end>  banana  black  color  hair  has  is  mango  pink  yellow
0      1       0      0      0     0    0   0      0     0       0
1      0       1      0      0     0    0   0      0     0       0
2      0       0      1      0     0    0   0      0     0       0
3      0       0      0      1     0    0   0      0     0       0
4      0       0      0      0     1    0   0      0     0       0
5      0       0      0      0     0    1   0      0     0       0
6      0       0      0      0     0    0   1      0     0       0
7      0       0      0      0     0    0   0      1     0       0
8      0       0      0      0     0    0   0      0     1       0
9      0       0      0      0     0    0   0      0     0       1


In [3]:
# now we create two list which stores [character_to_index] values and [index_to_character] values

char2idx = {value:index for index,value in enumerate(vocab)}
idx2char = np.array(vocab)

In [4]:
# max length of time step

seq_length = max(len(s1.split()),len(s2.split()),len(s3.split()))
seq_length

4

# Preparing Input data for single batch 

In [5]:
# For single batch  (1*10*4) or (10*4) loop over 4 i.e loop (1*10) (1*10) (1*10) (1*10)

##############################################################
# input  #       shape      #   target   #       shape       #
##############################################################
# mango  (1*10) onehotvector     is      (1*10) onehotvector #
# is     (1*10) onehotvector    yellow   (1*10) onehotvector #
# yellow (1*10) onehotvector    color    (1*10) onehotvector #
# color  (1*10) onehotvector    <end>    (1*10) onehotvector #
##############################################################

In [6]:
# right now we pass single sentence or single batch so the input size will become 
#(1*10*5) batchsize * vocab_size * sequence_size or timestep 
print([char2idx[char] for char in s1.split()])
print(one_hot_vector_vocab[[7,6,9,3]].T.shape)
one_hot_vector_vocab[[7,6,9,3]].T  # mango is yellow color OH


[7, 6, 9, 3]
(10, 4)


array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0]], dtype=uint8)

In [7]:
# Input array for single batch
np.array(s1.split()) # 4 words each represent in One Hot Encoding  dim = (10*4)

array(['mango', 'is', 'yellow', 'color'], dtype='<U6')

In [8]:
# prepare input data for single batch pass
x1 = one_hot_vector_vocab[[char2idx[char] for char in s1.split()]].T #s1 = ' mango is yellow color '
x2 = one_hot_vector_vocab[[char2idx[char] for char in s2.split()]].T #s2 = ' banana is pink color  '
x3 = one_hot_vector_vocab[[char2idx[char] for char in s3.split()]].T #s3 = ' hair has black color  '
print(x1.shape) # these are
print(x2.shape) # three
print(x3.shape) # different samples

print(x1[:,0].shape) # 0th time step means we are passing mango as an input
x1[:,0]

(10, 4)
(10, 4)
(10, 4)
(10,)


array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

# Preparing target variable for single batch

In [9]:
y1 = s1.split()[1:] # why I take from 1st index? bcz first input word predict 2nd so we expect 2 word from output
y2 = s2.split()[1:]
y3 = s3.split()[1:]
y1.append('<end>') ; y2.append('<end>') ; y3.append('<end>') ;
y1 = np.array(y1)
y2 = np.array(y2)
y3 = np.array(y3)
print(y1) # these are
print(y2) # three 
print(y3) # different samples
[char2idx[i] for i in y1] # this is expected targets

['is' 'yellow' 'color' '<end>']
['is' 'pink' 'color' '<end>']
['has' 'black' 'color' '<end>']


[6, 9, 3, 0]

In [10]:
y1 = np.array([char2idx[i] for i in y1]) # this is expected targets
y2 = np.array([char2idx[i] for i in y2]) # this is expected targets
y3 = np.array([char2idx[i] for i in y3]) # this is expected targets
y3

array([5, 2, 3, 0])

# Forward Propagation

## 1.Single timestep feedforward

In [11]:
def softmax(a):
    return np.exp(a)/np.exp(a).sum(axis=0)

def rnn_cell_forward(xt, a_prev, parameters):
    """
    Arguments:
    xt         --  (batch_unit_size , vocab_size)
    a_prev     --  (hidden_unit_size, vocab_size)

    parameters -- python dictionary containing:
                        Wax -- (batch_unit_size , hidden_unit_size)
                        Waa -- (hidden_unit_size, hidden_unit_size)
                        Wya -- (hidden_unit_size, output_unit_size)
                        ba --  (hidden_unit_size, 1)
                        by --  (output_unit_size, 1)
    Returns:
    a_next  -- (hidden_unit, vocab_size)
    yt_pred -- (output_unit, vocab_size)
    cache   -- (a_next, a_prev, xt, parameters)
    """
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Way = parameters["Way"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    a_next = np.tanh(np.dot(Waa.T, a_prev) + np.dot(Wax.T, xt) + ba) # current activation bias
    
    y_cap = softmax(np.dot(Way.T, a_next) + by)  # output of current time step
    
    cache = (a_prev, xt, a_next, parameters) # need during back propagation 
    
    return a_next, y_cap, cache

In [12]:
x1[:,0] # mango
x1[:,1] # is 
x1[:,2] # yellow
x1[:,3] # color

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [13]:
#           Wax.T  *  Xt               Way.T  * a_current
# (1*10)    (2*1)  * (1*10)             (1*2) *  (2*10)
# mango ---------------------> (2*10) --------------------->  is = (1*10)


In [14]:
# we create 1 hidden layer with 2 neurons
np.random.seed(1)
xt     = x1[:,0].reshape(1,-1)  # (1*10)  ==> batch_size    * vocab_size 
Wax    = np.random.randn(1,2)   # (1*2)   ==> batch_size    * hidden_neuron

a_prev = np.random.randn(2,10)  # (2*10)  ==> hidden_neuron * vocab_size
Waa    = np.random.randn(2,2)   # (2*2)   ==> hidden_neuron * hidden_neuron
ba     = np.random.randn(2,1)   # (2*1)   ==> hidden nueron * 1

Way    = np.random.randn(2,1)   # (2*1)   ==> hidden_neuron * output_neuron(or batch_size)
by     = np.random.randn(1,1)   # (1*1)   ==> output_neuron * 1

# in sentiment anlysis output_neron will be no. of labels 

parameters = {"Waa": Waa, "Wax": Wax, "Way": Way, "ba": ba, "by": by}

a_next, y_cap, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next = ", a_next.shape)
print("a_next.shape = ", a_next.shape)
print("y_cap.shape = ", y_cap.shape)

a_next =  (2, 10)
a_next.shape =  (2, 10)
y_cap.shape =  (1, 10)


In [16]:
# dont expect good output because we dont train this RNN. I just explain whole code with short sample 
y_cap  # index with max value would be next possible word but here all are possible. lol

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

## 2.Multi timestep feedforward 
#### Don't think that it's a multi neuron like NN. It just a single neuron where activations are feed in loop 

You can see an RNN as the repetition of the cell you've just built. If your input sequence of data is carried over 4 time steps, then you will copy the RNN cell 4 times. Each cell takes as input the hidden state from the previous cell ($a^{\langle t-1 \rangle}$) and the current time-step's input data ($x^{\langle t \rangle}$). It outputs a hidden state ($a^{\langle t \rangle}$) and a prediction ($y^{\langle t \rangle}$) for this time-step.


<img src="images/rnn.png" style="width:800px;height:300px;">
<caption><center> **Figure 3**: Basic RNN. The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  is carried over $T_x$ time steps. The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>

In [71]:
def rnn_forward(xt, a_prev, parameters):
    """
    Arguments:
    xt         --  (batch_unit_size , vocab_size , timestep)
    a_prev     --  (hidden_unit_size, vocab_size)

    parameters -- python dictionary containing:
                        Wax -- (batch_unit_size , hidden_unit_size)
                        Waa -- (hidden_unit_size, hidden_unit_size)
                        Way -- (hidden_unit_size, output_unit_size)
                        ba --  (hidden_unit_size, 1)
                        by --  (output_unit_size, 1)
    Returns:
    a_next  -- (hidden_unit, vocab_size)
    yt_pred -- (output_unit, vocab_size)
    cache   -- (a_next, a_prev, xt, parameters)
    """
    
    caches = []  # store outputs which we get using single timestep  [need during backpropagation]
    
    hidden_neuron , vocab_size         =  a_prev.shape
    batch_size , vocab_size , timestep =  xt.shape
    hidden_neuron, output_neuron       = parameters["Way"].shape
    
    activations = np.zeros([hidden_neuron,vocab_size,timestep])  # store activations during each timesteps
    y_pred      = np.zeros([output_neuron,vocab_size,timestep])  # store output at each timesteps
    
    a_next = a_prev  # why? you will get soon
    
    # loop over all time-steps
    for t in range(timestep):
        
        a_next, y_cap, cache = rnn_cell_forward(xt[:,:,t], a_next, parameters)
        activations[:,:,t]   = a_next
        y_pred[:,:,t]        = y_cap
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, xt)
    
    return activations, y_pred, caches

In [72]:
np.random.seed(1)
xt     = x1.reshape(1,10,4)     # (1*10*4) ==> batch_size    * vocab_size * timestep 
Wax    = np.random.randn(1,2)   # (1*2)    ==> batch_size    * hidden_neuron

a_prev = np.random.randn(2,10)  # (2*10)   ==> hidden_neuron * vocab_size
Waa    = np.random.randn(2,2)   # (2*2)    ==> hidden_neuron * hidden_neuron
ba     = np.random.randn(2,1)   # (2*1)    ==> hidden nueron * 1

Way    = np.random.randn(2,1)   # (2*1)    ==> hidden_neuron * output_neuron(or batch_size)
by     = np.random.randn(1,1)   # (1*1)    ==> output_neuron * 1

parameters = {"Waa": Waa, "Wax": Wax, "Way": Way, "ba": ba, "by": by}

activations, y_caps, caches = rnn_forward(xt, a_prev, parameters)
print("activations.shape = ", activations.shape)
print("y_caps.shape      = ", y_caps.shape)
print("len(caches)       = ", len(caches))

activations.shape =  (2, 10, 4)
y_caps.shape      =  (1, 10, 4)
len(caches)       =  2


In [73]:
caches    # len = 2 it is tuple which contains (cache , xt)
caches[0] # len = 4 (4 time step data)
caches[1] # len = (1*10*4) input data

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0]]], dtype=uint8)

In [74]:
caches[0][0] # first time step type is tuple (a_prev , xt , a_next , parameters)

(array([[-0.52817175, -1.07296862,  0.86540763, -2.3015387 ,  1.74481176,
         -0.7612069 ,  0.3190391 , -0.24937038,  1.46210794, -2.06014071],
        [-0.3224172 , -0.38405435,  1.13376944, -1.09989127, -0.17242821,
         -0.87785842,  0.04221375,  0.58281521, -1.10061918,  1.14472371]]),
 array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]], dtype=uint8),
 array([[-0.71116469, -0.89293958,  0.93269476, -0.99660724,  0.86040002,
         -0.92167025,  0.20004648,  0.94697741,  0.20105657, -0.73935847],
        [-0.7533805 , -0.83738047, -0.85544148, -0.87173307,  0.05881461,
         -0.61570329, -0.66644319, -0.96873478,  0.5016103 , -0.99191882]]),
 {'Waa': array([[ 0.90159072,  0.50249434],
         [ 0.90085595, -0.68372786]]),
  'Wax': array([[ 1.62434536, -0.61175641]]),
  'Way': array([[-0.26788808],
         [ 0.53035547]]),
  'ba': array([[-0.12289023],
         [-0.93576943]]),
  'by': array([[-0.69166075]])})

In [75]:
caches[0][1] # second time step type is tuple (a_prev , xt , a_next , parameters)

(array([[-0.71116469, -0.89293958,  0.93269476, -0.99660724,  0.86040002,
         -0.92167025,  0.20004648,  0.94697741,  0.20105657, -0.73935847],
        [-0.7533805 , -0.83738047, -0.85544148, -0.87173307,  0.05881461,
         -0.61570329, -0.66644319, -0.96873478,  0.5016103 , -0.99191882]]),
 array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], dtype=uint8),
 array([[-0.89425136, -0.93316138, -0.05256234, -0.94749833,  0.6080502 ,
         -0.90667597,  0.79373466, -0.14085196,  0.47014708, -0.93325815],
        [-0.65156786, -0.67065145,  0.11725177, -0.68609008, -0.49573571,
         -0.75216843, -0.75793221,  0.19971175, -0.82672644, -0.55742553]]),
 {'Waa': array([[ 0.90159072,  0.50249434],
         [ 0.90085595, -0.68372786]]),
  'Wax': array([[ 1.62434536, -0.61175641]]),
  'Way': array([[-0.26788808],
         [ 0.53035547]]),
  'ba': array([[-0.12289023],
         [-0.93576943]]),
  'by': array([[-0.69166075]])})

In [76]:
caches[0][2] # third time step type is tuple (a_prev , xt , a_next , parameters)

(array([[-0.89425136, -0.93316138, -0.05256234, -0.94749833,  0.6080502 ,
         -0.90667597,  0.79373466, -0.14085196,  0.47014708, -0.93325815],
        [-0.65156786, -0.67065145,  0.11725177, -0.68609008, -0.49573571,
         -0.75216843, -0.75793221,  0.19971175, -0.82672644, -0.55742553]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8),
 array([[-0.90801696, -0.91676773, -0.06456306, -0.92094498, -0.02126107,
         -0.92432426, -0.08981151, -0.06985556, -0.41676567,  0.15657936],
        [-0.73505248, -0.73802812, -0.77881437, -0.73649832, -0.28331189,
         -0.70495873, -0.01870071, -0.81545373, -0.13346623, -0.92682048]]),
 {'Waa': array([[ 0.90159072,  0.50249434],
         [ 0.90085595, -0.68372786]]),
  'Wax': array([[ 1.62434536, -0.61175641]]),
  'Way': array([[-0.26788808],
         [ 0.53035547]]),
  'ba': array([[-0.12289023],
         [-0.93576943]]),
  'by': array([[-0.69166075]])})

In [77]:
caches[0][3] # forth time step type is tuple (a_prev , xt , a_next , parameters)

(array([[-0.90801696, -0.91676773, -0.06456306, -0.92094498, -0.02126107,
         -0.92432426, -0.08981151, -0.06985556, -0.41676567,  0.15657936],
        [-0.73505248, -0.73802812, -0.77881437, -0.73649832, -0.28331189,
         -0.70495873, -0.01870071, -0.81545373, -0.13346623, -0.92682048]]),
 array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=uint8),
 array([[-0.92222754, -0.92379244, -0.70776899,  0.00766064, -0.37762112,
         -0.92035127, -0.21719476, -0.72612332, -0.55034504, -0.67324321],
        [-0.71113035, -0.71229625, -0.41008627, -0.90635722, -0.63678359,
         -0.72506242, -0.74787389, -0.39129058, -0.78333237, -0.21975267]]),
 {'Waa': array([[ 0.90159072,  0.50249434],
         [ 0.90085595, -0.68372786]]),
  'Wax': array([[ 1.62434536, -0.61175641]]),
  'Way': array([[-0.26788808],
         [ 0.53035547]]),
  'ba': array([[-0.12289023],
         [-0.93576943]]),
  'by': array([[-0.69166075]])})

# Cost Function

<img src="images/costfunction.jpeg" style="width:500;height:300px;">

In [78]:
print(np.argmax(y_caps[:,:,0]))
print(np.argmax(y_caps[:,:,1]))
print(np.argmax(y_caps[:,:,2]))
print(np.argmax(y_caps[:,:,3]))

0
0
0
0


In [82]:
y_pred = np.array([np.argmax(y_caps[:,:,0]),np.argmax(y_caps[:,:,1]),np.argmax(y_caps[:,:,2]),np.argmax(y_caps[:,:,3])])

In [83]:
#y1
np.argmax(y_caps[:,:,0])
np.argmax(y_caps[:,:,1])
np.argmax(y_caps[:,:,2])
np.argmax(y_caps[:,:,3])

0

In [84]:
def costfunction(y_pred, y_actual):
    return (1/4)*(np.dot(y_actual,np.log(y_pred)))
#costfunction(y_pred,y1)
#np.log(y_pred)

# Backpropagation 

###  Basic RNN  backward pass

We will start by computing the backward pass for the basic RNN-cell.

<img src="images/rnn_cell_backprop.png" style="width:500;height:300px;"> <br>
<caption><center> **Figure 6**: RNN-cell's backward pass. Just like in a fully-connected neural network, the derivative of the cost function $J$ backpropagates through the RNN by following the chain-rule from calculus. The chain-rule is also used to calculate $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ to update the parameters $(W_{ax}, W_{aa}, b_a)$. </center></caption>

In [85]:
def rnn_cell_backward(da_next, cache):
    """
    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache   -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                        dx      -- Gradients of input data, of shape              (batch_size ,vocab_size, timestep)
                        da_prev -- Gradients of previous hidden state, of shape   (hidden_unit,vocab_size)
                        dWax    -- Gradients of input-to-hidden weights, of shape (batch_size ,hidden_unit)
                        dWaa    -- Gradients of hidden-to-hidden weights, of shape(hidden_unit,hidden_unit)
                        dba     -- Gradients of bias vector, of shape             (hidden_unit, 1)
    """
    
    # Retrieve values from cache
    (a_prev, xt, a_next, parameters) = cache # (2*10),(1*10),(2*10),
                                            # (Waa,Wax,Way,ba,by) = (2*2),(1*2),(2*1),(2*1),(1*1)
    
    # Retrieve values from parameters
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Way = parameters["Way"]
    ba = parameters["ba"]
    by = parameters["by"]

    # compute the gradient of tanh with respect to a_next 
    da_dtanh = (1 - np.square(a_next))  # a_next = np.tanh(x) we calculate this in forward pass  (2*10)
    
                    #(1*2)  (2*10)
    da_dxt    = np.dot(Wax , da_dtanh)  # (1*10)
                    #(2*2)  (2*10)   
    da_da_prev = np.dot(Waa , da_dtanh)  # (2*10)
    
                    #(1*10) (2*10).T
    da_dWax   = np.dot(xt, da_dtanh.T)     # (1*2)
                    #(2*10)  (2*10).T
    da_dWaa   = np.dot(a_prev, da_dtanh.T) # (2*2)

                    #(2*10)
    da_dba = np.sum(da_dtanh,axis = 1).reshape(-1,1)  # (2*1)
    
    # Store the gradients in a python dictionary
    gradients = {"da_dxt": da_dxt, "da_da_prev": da_da_prev, "da_dWax": da_dWax, "da_dWaa": da_dWaa, "da_dba": da_dba}
    
    return gradients

In [86]:
np.random.seed(1)
xt     = x1.reshape(1,10,4)[:,:,0] # (1*10*4) ==> batch_size    * vocab_size * timestep 
Wax    = np.random.randn(1,2)   # (1*2)    ==> batch_size    * hidden_neuron

a_prev = np.random.randn(2,10)  # (2*10)   ==> hidden_neuron * vocab_size
Waa    = np.random.randn(2,2)   # (2*2)    ==> hidden_neuron * hidden_neuron
ba     = np.random.randn(2,1)   # (2*1)    ==> hidden nueron * 1

Way    = np.random.randn(2,1)   # (2*1)    ==> hidden_neuron * output_neuron(or batch_size)
by     = np.random.randn(1,1)   # (1*1)    ==> output_neuron * 1

parameters = {"Waa": Waa, "Wax": Wax, "Way": Way, "ba": ba, "by": by}

activations, y_caps, caches = rnn_cell_forward(xt, a_prev, parameters)

da_next = np.random.randn(2,10)
gradients = rnn_cell_backward(da_next , caches) # da_dxt, da_da_prev, da_dWax, da_dWaa, da_dba

print("gradients[\"dxt\"].shape =", gradients["da_dxt"].shape)
print("gradients[\"da_prev\"].shape =", gradients["da_da_prev"].shape)
print("gradients[\"dWax\"].shape =", gradients["da_dWax"].shape)
print("gradients[\"dWaa\"].shape =", gradients["da_dWaa"].shape)
print("gradients[\"dba\"].shape =", gradients["da_dba"].shape)

gradients["dxt"].shape = (1, 10)
gradients["da_prev"].shape = (2, 10)
gradients["dWax"].shape = (1, 2)
gradients["dWaa"].shape = (2, 2)
gradients["dba"].shape = (2, 1)


In [125]:
def rnn_backward(da, caches):
    
    """
    Arguments:
    da     -- Upstream gradients of all hidden states, of shape (batch_size, vocab_size, timestep)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                    dx  -- Gradient w.r.t. the input data, numpy-array of shape (batch_size, vocab_size, timestep)
                    da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (hidden_unit, vocab_size)
                    dWax-- Gradient w.r.t the input's weight matrix, numpy-array of shape (batch_size, hidden_unit)
                    dWaa-- Gradient w.r.t the hidden state's weight matrix, numpy-arrayof shape (hidden_unit,hidden_unit)
                    dba -- Gradient w.r.t the bias, of shape (hidden_unit, 1)
    """
    
    # Retrieve values from the first cache (t=1) of caches
    (caches, x) = caches # all time step data ===> where x contains a_next of all timesteps 
                                             # caches contains a_prev,xt,a_next,parameters for all timesteps
    (a0, x1, a1, parameters) = caches[0] # first time step data
    
    
    batch_size, vocab_size =  x1.shape
    timestep = len(caches) 
    hidden_unit, vocab_size = a0.shape
    
    dx   = np.zeros([batch_size,vocab_size,timestep])
    dWax = 0
    dWaa = 0
    dba  = 0
    da0  = 0
    da_prevt = 0
    
    # Loop through all the time steps
    for t in reversed(range(timestep)):
        # Compute gradients at time step t. 
        
        gradients = rnn_cell_backward(da[:,:,t] , caches[t])
        da_dxtt, da_da_prevt = gradients["da_dxt"], gradients["da_da_prev"],
        da_dWaxt, da_dWaat, da_dbat =  gradients["da_dWax"], gradients["da_dWaa"], gradients["da_dba"]
        
        dx[:, :, t] = da_dxtt
        dWax += da_dWaxt
        dWaa += da_dWaat
        dba += da_dbat
        da_prevt += da_da_prevt
    
    da0 = da_prevt
    
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
    
    return gradients

In [126]:
np.random.seed(1)
xt  = x1.reshape(1,10,4)     # (1*10*4) ==> batch_size    * vocab_size * timestep 
Wax = np.random.randn(1,2)   # (1*2)    ==> batch_size    * hidden_neuron

a0  = np.random.randn(2,10)  # (2*10)   ==> hidden_neuron * vocab_size
Waa = np.random.randn(2,2)   # (2*2)    ==> hidden_neuron * hidden_neuron
ba  = np.random.randn(2,1)   # (2*1)    ==> hidden nueron * 1

Way = np.random.randn(2,1)   # (2*1)    ==> hidden_neuron * output_neuron(or batch_size)
by  = np.random.randn(1,1)   # (1*1)    ==> output_neuron * 1

parameters = {"Waa": Waa, "Wax": Wax, "Way": Way, "ba": ba, "by": by}

activations, y_caps, caches = rnn_forward(xt, a0, parameters)
print("activations.shape = ", activations.shape)
print("y_caps.shape      = ", y_caps.shape)
print("len(caches)       = ", len(caches))

da = np.random.randn(2, 10, 4)
gradients = rnn_backward(da, caches) # caches = da = [d(a_t+1)*4   (at, at-1, xt, parameter)]

#print("gradients[\"dx\"].shape =",   gradients["dx"].shape)
#print("gradients[\"da0\"].shape =",  gradients["da0"].shape)
#print("gradients[\"dWax\"].shape =", gradients["dWax"].shape)
#print("gradients[\"dWaa\"].shape =", gradients["dWaa"].shape)
#print("gradients[\"dba\"].shape =",  gradients["dba"].shape)


activations.shape =  (2, 10, 4)
y_caps.shape      =  (1, 10, 4)
len(caches)       =  2


In [127]:
gradients

{'dWaa': array([[  1.11129101,   0.11771204],
        [ -8.20854707, -12.76779123]]),
 'dWax': array([[2.44864329, 0.80661209]]),
 'da0': array([[ 1.90505448,  1.4781936 ,  3.61022234,  1.84297594,  4.117271  ,
          1.58663874,  4.16936769,  3.40748605,  4.16377087,  2.80102035],
        [-0.42290677, -0.65387985,  0.6667619 ,  0.17487511,  0.24220396,
         -0.82478276,  1.29425499,  0.7916581 ,  1.27471698,  0.66654395]]),
 'dba': array([[20.10714295],
        [21.79842979]]),
 'dx': array([[[ 0.53828984, -0.02666099,  0.00385796, -0.05955356],
         [ 0.14639895, -0.12672326, -0.01939849, -0.06323078],
         [ 0.04721041,  1.01651162,  1.37688001,  0.30177389],
         [-0.13586802, -0.15770639, -0.03324789,  1.51504137],
         [-0.18777859,  0.56236931,  1.0609577 ,  1.02902418],
         [-0.13534247,  0.02338314, -0.07118904, -0.04169721],
         [ 1.21929442,  0.34065781,  0.99970075,  1.27812755],
         [ 0.13003192,  1.00476289,  1.41145894,  0.24980961]

## 3.2 - LSTM backward pass

### 3.2.1 One Step backward

The LSTM backward pass is slighltly more complicated than the forward one. We have provided you with all the equations for the LSTM backward pass below. (If you enjoy calculus exercises feel free to try deriving these from scratch yourself.) 

### 3.2.2 gate derivatives

$$d \Gamma_o^{\langle t \rangle} = da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*(1-\Gamma_o^{\langle t \rangle})\tag{7}$$

$$d\tilde c^{\langle t \rangle} = dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * i_t * da_{next} * \tilde c^{\langle t \rangle} * (1-\tanh(\tilde c)^2) \tag{8}$$

$$d\Gamma_u^{\langle t \rangle} = dc_{next}*\tilde c^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * \tilde c^{\langle t \rangle} * da_{next}*\Gamma_u^{\langle t \rangle}*(1-\Gamma_u^{\langle t \rangle})\tag{9}$$

$$d\Gamma_f^{\langle t \rangle} = dc_{next}*\tilde c_{prev} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * c_{prev} * da_{next}*\Gamma_f^{\langle t \rangle}*(1-\Gamma_f^{\langle t \rangle})\tag{10}$$

### 3.2.3 parameter derivatives 

$$ dW_f = d\Gamma_f^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{11} $$
$$ dW_u = d\Gamma_u^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{12} $$
$$ dW_c = d\tilde c^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{13} $$
$$ dW_o = d\Gamma_o^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{14}$$

To calculate $db_f, db_u, db_c, db_o$ you just need to sum across the horizontal (axis= 1) axis on $d\Gamma_f^{\langle t \rangle}, d\Gamma_u^{\langle t \rangle}, d\tilde c^{\langle t \rangle}, d\Gamma_o^{\langle t \rangle}$ respectively. Note that you should have the `keep_dims = True` option.

Finally, you will compute the derivative with respect to the previous hidden state, previous memory state, and input.

$$ da_{prev} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c^{\langle t \rangle} + W_o^T * d\Gamma_o^{\langle t \rangle} \tag{15}$$
Here, the weights for equations 13 are the first n_a, (i.e. $W_f = W_f[:n_a,:]$ etc...)

$$ dc_{prev} = dc_{next}\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh(c_{next})^2)*\Gamma_f^{\langle t \rangle}*da_{next} \tag{16}$$
$$ dx^{\langle t \rangle} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c_t + W_o^T * d\Gamma_o^{\langle t \rangle}\tag{17} $$
where the weights for equation 15 are from n_a to the end, (i.e. $W_f = W_f[n_a:,:]$ etc...)

**Exercise:** Implement `lstm_cell_backward` by implementing equations $7-17$ below. Good luck! :)

In [ ]:
]:
def softmax(y_linear, temperature=1.0):
    lin = (y_linear-nd.max(y_linear, axis=1).reshape((-1,1))) / temperature # shift each row of y_linear by its max
    exp = nd.exp(lin)
    partition =nd.sum(exp, axis=1).reshape((-1,1))
    return exp / partition

 def cross_entropy(yhat, y):
     return - nd.sum(y * nd.log(yhat))

def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = 0.
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)

In [ ]:
def lstm_cell_backward(da_next, dc_next, cache):
    """
    Implement the backward pass for the LSTM-cell (single time-step).

    Arguments:
    da_next -- Gradients of next hidden state, of shape (n_a, m)
    dc_next -- Gradients of next cell state, of shape (n_a, m)
    cache -- cache storing information from the forward pass

    Returns:
    gradients -- python dictionary containing:
                        dxt -- Gradient of input data at time-step t, of shape (n_x, m)
                        da_prev -- Gradient w.r.t. the previous hidden state, numpy array of shape (n_a, m)
                        dc_prev -- Gradient w.r.t. the previous memory state, of shape (n_a, m, T_x)
                        dWf -- Gradient w.r.t. the weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        dWi -- Gradient w.r.t. the weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        dWc -- Gradient w.r.t. the weight matrix of the memory gate, numpy array of shape (n_a, n_a + n_x)
                        dWo -- Gradient w.r.t. the weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                        dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                        dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                        dbo -- Gradient w.r.t. biases of the output gate, of shape (n_a, 1)
    """

    # Retrieve information from "cache"
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache
    
    ### START CODE HERE ###
    # Retrieve dimensions from xt's and a_next's shape (≈2 lines)
    n_x, m = None
    n_a, m = None
    
    # Compute gates related derivatives, you can find their values can be found by looking carefully at equations (7) to (10) (≈4 lines)
    dot = None
    dcct = None
    dit = None
    dft = None
    
    # Code equations (7) to (10) (≈4 lines)
    dit = None
    dft = None
    dot = None
    dcct = None

    # Compute parameters related derivatives. Use equations (11)-(14) (≈8 lines)
    dWf = None
    dWi = None
    dWc = None
    dWo = None
    dbf = None
    dbi = None
    dbc = None
    dbo = None

    # Compute derivatives w.r.t previous hidden state, previous memory state and input. Use equations (15)-(17). (≈3 lines)
    da_prev = None
    dc_prev = None
    dxt = None
    ### END CODE HERE ###
    
    # Save gradients in dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

    return gradients

In [ ]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
c_prev = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)

da_next = np.random.randn(5,10)
dc_next = np.random.randn(5,10)
gradients = lstm_cell_backward(da_next, dc_next, cache)
print("gradients[\"dxt\"][1][2] =", gradients["dxt"][1][2])
print("gradients[\"dxt\"].shape =", gradients["dxt"].shape)
print("gradients[\"da_prev\"][2][3] =", gradients["da_prev"][2][3])
print("gradients[\"da_prev\"].shape =", gradients["da_prev"].shape)
print("gradients[\"dc_prev\"][2][3] =", gradients["dc_prev"][2][3])
print("gradients[\"dc_prev\"].shape =", gradients["dc_prev"].shape)
print("gradients[\"dWf\"][3][1] =", gradients["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients["dWf"].shape)
print("gradients[\"dWi\"][1][2] =", gradients["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients["dWi"].shape)
print("gradients[\"dWc\"][3][1] =", gradients["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients["dWc"].shape)
print("gradients[\"dWo\"][1][2] =", gradients["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients["dWo"].shape)
print("gradients[\"dbf\"][4] =", gradients["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients["dbf"].shape)
print("gradients[\"dbi\"][4] =", gradients["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients["dbi"].shape)
print("gradients[\"dbc\"][4] =", gradients["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients["dbc"].shape)
print("gradients[\"dbo\"][4] =", gradients["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients["dbo"].shape)

**Expected Output**:

<table>
    <tr>
        <td>
            **gradients["dxt"][1][2]** =
        </td>
        <td>
           3.23055911511
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dxt"].shape** =
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"][2][3]** =
        </td>
        <td>
           -0.0639621419711
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            **gradients["dc_prev"][2][3]** =
        </td>
        <td>
           0.797522038797
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dc_prev"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"][3][1]** = 
        </td>
        <td>
           -0.147954838164
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"].shape** =
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"][1][2]** = 
        </td>
        <td>
           1.05749805523
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWc"][3][1]** = 
        </td>
        <td>
           2.30456216369
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWc"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWo"][1][2]** = 
        </td>
        <td>
           0.331311595289
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWo"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbf"][4]** = 
        </td>
        <td>
           [ 0.18864637]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbf"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbi"][4]** = 
        </td>
        <td>
           [-0.40142491]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbi"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"][4]** = 
        </td>
        <td>
           [ 0.25587763]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"][4]** = 
        </td>
        <td>
           [ 0.13893342]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### 3.3 Backward pass through the LSTM RNN

This part is very similar to the `rnn_backward` function you implemented above. You will first create variables of the same dimension as your return variables. You will then iterate over all the time steps starting from the end and call the one step function you implemented for LSTM at each iteration. You will then update the parameters by summing them individually. Finally return a dictionary with the new gradients. 

**Instructions**: Implement the `lstm_backward` function. Create a for loop starting from $T_x$ and going backward. For each step call `lstm_cell_backward` and update the your old gradients by adding the new gradients to them. Note that `dxt` is not updated but is stored.

In [ ]:
def lstm_backward(da, caches):
    
    """
    Implement the backward pass for the RNN with LSTM-cell (over a whole sequence).

    Arguments:
    da -- Gradients w.r.t the hidden states, numpy-array of shape (n_a, m, T_x)
    caches -- cache storing information from the forward pass (lstm_forward)

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient of inputs, of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t. the previous hidden state, numpy array of shape (n_a, m)
                        dWf -- Gradient w.r.t. the weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        dWi -- Gradient w.r.t. the weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        dWc -- Gradient w.r.t. the weight matrix of the memory gate, numpy array of shape (n_a, n_a + n_x)
                        dWo -- Gradient w.r.t. the weight matrix of the save gate, numpy array of shape (n_a, n_a + n_x)
                        dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                        dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                        dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                        dbo -- Gradient w.r.t. biases of the save gate, of shape (n_a, 1)
    """

    # Retrieve values from the first cache (t=1) of caches.
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]
    
    ### START CODE HERE ###
    # Retrieve dimensions from da's and x1's shapes (≈2 lines)
    n_a, m, T_x = None
    n_x, m = None
    
    # initialize the gradients with the right sizes (≈12 lines)
    dx = None
    da0 = None
    da_prevt = None
    dc_prevt = None
    dWf = None
    dWi = None
    dWc = None
    dWo = None
    dbf = None
    dbi = None
    dbc = None
    dbo = None
    
    # loop back over the whole sequence
    for t in reversed(range(None)):
        # Compute all gradients using lstm_cell_backward
        gradients = None
        # Store or add the gradient to the parameters' previous step's gradient
        dx[:,:,t] = None
        dWf = None
        dWi = None
        dWc = None
        dWo = None
        dbf = None
        dbi = None
        dbc = None
        dbo = None
    # Set the first activation's gradient to the backpropagated gradient da_prev.
    da0 = None
    
    ### END CODE HERE ###

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}
    
    return gradients

In [ ]:
np.random.seed(1)
x = np.random.randn(3,10,7)
a0 = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a, y, c, caches = lstm_forward(x, a0, parameters)

da = np.random.randn(5, 10, 4)
gradients = lstm_backward(da, caches)

print("gradients[\"dx\"][1][2] =", gradients["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients["da0"].shape)
print("gradients[\"dWf\"][3][1] =", gradients["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients["dWf"].shape)
print("gradients[\"dWi\"][1][2] =", gradients["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients["dWi"].shape)
print("gradients[\"dWc\"][3][1] =", gradients["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients["dWc"].shape)
print("gradients[\"dWo\"][1][2] =", gradients["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients["dWo"].shape)
print("gradients[\"dbf\"][4] =", gradients["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients["dbf"].shape)
print("gradients[\"dbi\"][4] =", gradients["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients["dbi"].shape)
print("gradients[\"dbc\"][4] =", gradients["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients["dbc"].shape)
print("gradients[\"dbo\"][4] =", gradients["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients["dbo"].shape)

**Expected Output**:

<table>
    <tr>
        <td>
            **gradients["dx"][1][2]** =
        </td>
        <td>
           [-0.00173313  0.08287442 -0.30545663 -0.43281115]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dx"].shape** =
        </td>
        <td>
           (3, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"][2][3]** =
        </td>
        <td>
           -0.095911501954
        </td>
    </tr>
        <tr>
        <td>
            **gradients["da0"].shape** =
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"][3][1]** = 
        </td>
        <td>
           -0.0698198561274
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWf"].shape** =
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"][1][2]** = 
        </td>
        <td>
           0.102371820249
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWi"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWc"][3][1]** = 
        </td>
        <td>
           -0.0624983794927
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWc"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dWo"][1][2]** = 
        </td>
        <td>
           0.0484389131444
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dWo"].shape** = 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbf"][4]** = 
        </td>
        <td>
           [-0.0565788]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbf"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            **gradients["dbi"][4]** = 
        </td>
        <td>
           [-0.06997391]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbi"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"][4]** = 
        </td>
        <td>
           [-0.27441821]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbc"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"][4]** = 
        </td>
        <td>
           [ 0.16532821]
        </td>
    </tr>
        <tr>
        <td>
            **gradients["dbo"].shape** = 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### Congratulations !

Congratulations on completing this assignment. You now understand how recurrent neural networks work! 

Let's go on to the next exercise, where you'll use an RNN to build a character-level language model.
